In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import numpy as np
from pprint import pformat
import pandas as pd
import matplotlib.pyplot as plt
import datetime

## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (       
        ('printlog', False),
        ('rsiperiod',14),
        ('rsicrossover_high',50),
        ('rsicrossover_low',50)
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low
        self.dataopen = self.datas[0].open

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None


        # Add a RSI indicator
        self.rsi = bt.indicators.RelativeStrengthIndex(
            self.datas[0], period=self.params.rsiperiod)
        self.buysig = bt.And(self.datahigh[0] > self.dataclose[-1], self.rsi>self.params.rsicrossover_high)
        self.sellsig = bt.Or(self.rsi<self.params.rsicrossover_low, self.dataclose[0] < self.datalow[-1])


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.buysig:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.sellsig:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(RSI Period: %2d, RSI Crossover Lo/High: %2d/%2d) Ending Value %.2f' %
                 (self.params.rsiperiod, self.params.rsicrossover_low, self.params.rsicrossover_high, self.broker.getvalue()), doprint=True)

## Analyzers

In [ ]:
class DecisiveAnalyzer(bt.Analyzer):
    
    def __init__(self):
        self.rsiperiod = self.strategy.params.rsiperiod
        self.rsicrossover_high = self.strategy.params.rsicrossover_high
        self.rsicrossover_low = self.strategy.params.rsicrossover_low
        self.equity = []
        self.equitydf = pd.DataFrame()

    def start(self):
        # Not needed ... but could be used
        self.init_cash = self.strategy.broker.cash 
        self.num_trades = 0
        self.first_trade_open = None

    def next(self):
        # Not needed ... but could be used
        pass

    def notify_trade(self, trade):
        if not self.first_trade_open:
            self.first_trade_open = self.strategy.datetime.datetime()
            self.equitydf.at[self.first_trade_open, 'equity'] = self.init_cash

        if trade.isclosed:
            self.num_trades += 1
            self.equitydf.at[self.strategy.datetime.datetime(), 'equity'] = self.strategy.broker.getvalue()
            
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        # Add trade profit to dataframe
        self.equitydf['net_profit'] = self.equitydf['equity'].diff()

        outp = {
            'params': (self.rsicrossover_high,self.rsicrossover_low,self.rsiperiod),
            'profit': self.final_val - self.init_cash,
            'num_trades': self.num_trades,
            'equitydf': self.equitydf,
        }
        
        return outp
    

In [ ]:
def best_result_from_cerebro_opti_run(result):
    params  = []
    n_trades = []
    profit   = []
    trades   = []
    for res in result:
        r = res[0].analyzers.decisive.get_analysis()
        params.append(r['params'])
        n_trades.append(r['ntrade'])
        profit.append(r['profit'])
        trades.append(r['trades'])

    prof_ind = np.argmax(profit) 
    best_params = params[prof_ind]
    best_profit = profit[prof_ind]
    best_ntrades = n_trades[prof_ind]
    best_trades = trades[prof_ind]
    
    print('best:{} profit:{} trades:{}'.format(best_params, best_profit, best_ntrades))
    return (best_params, best_profit, best_ntrades, best_trades)

In [ ]:
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()
df['2017-01-01':'2017-04-01']['close'].plot()

## In-sample Parameters

In [ ]:
periods = [
{'run': 0, 'oos': ('2017-04-01', '2017-05-01'), 'is': ('2017-01-01', '2017-04-01')},
{'run': 1, 'oos': ('2017-05-01', '2017-06-01'), 'is': ('2017-02-01', '2017-05-01')},
{'run': 2, 'oos': ('2017-06-01', '2017-07-01'), 'is': ('2017-03-01', '2017-06-01')},
{'run': 3, 'oos': ('2017-07-01', '2017-08-01'), 'is': ('2017-04-01', '2017-07-01')},
{'run': 4, 'oos': ('2017-08-01', '2017-09-01'), 'is': ('2017-05-01', '2017-08-01')},
{'run': 5, 'oos': ('2017-09-01', '2017-10-01'), 'is': ('2017-06-01', '2017-09-01')},
{'run': 6, 'oos': ('2017-10-01', '2017-11-01'), 'is': ('2017-07-01', '2017-10-01')},
{'run': 7, 'oos': ('2017-11-01', '2017-12-01'), 'is': ('2017-08-01', '2017-11-01')},
{'run': 8, 'oos': ('2017-12-01', '2018-01-01'), 'is': ('2017-09-01', '2017-12-01')},
{'run': 9, 'oos': ('2018-01-01', '2018-02-01'), 'is': ('2017-10-01', '2018-01-01')},
{'run': 10, 'oos': ('2018-02-01', '2018-03-01'), 'is': ('2017-11-01', '2018-02-01')},
{'run': 11, 'oos': ('2018-03-01', '2018-04-01'), 'is': ('2017-12-01', '2018-03-01')},
{'run': 12, 'oos': ('2018-04-01', '2018-05-01'), 'is': ('2018-01-01', '2018-04-01')},
{'run': 13, 'oos': ('2018-05-01', '2018-06-01'), 'is': ('2018-02-01', '2018-05-01')},

]

In [ ]:
start_date = periods[0]['is'][0]
end_date = periods[-1]['is'][1]
print('Start/End: {} - {}'.format(start_date, end_date))

cerebro = bt.Cerebro()

strats = cerebro.optstrategy(
    TestStrategy,
    rsiperiod=range(6,15,2),rsicrossover_high=range(50,71,10),rsicrossover_low=range(30,51,10))

cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

cerebro.optreturn = False
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.broker.setcommission(commission=0.0)

data = bt.feeds.PandasData(dataname = df[start_date:end_date])
cerebro.adddata(data)

results = cerebro.run(maxcpus=1)   

## Out-of-sample Parameters

In [ ]:
# Get in-sample and out-of-sample best parameters with correct pre-allocation
for period in periods:
    # Get the insample time period
    insample_start = period['is'][0]
    insample_end = period['is'][1]
    print('In-Sample Period: {} - {}'.format(insample_start, insample_end))
    
    # Put the best parameters and best profit into a list
    period_netprofit = []
    for result in results:
        r = result[0].analyzers.decisive.get_analysis()
        df = r['equitydf']
        rsicrossover_high= r['params'][0]
        rsicrossover_low = r['params'][1]
        rsiperiod = r['params'][2]
        period_profit = df[insample_start:insample_end]['net_profit'].sum()
        print('\t(RSI Period: {}, RSI Crossover Lo/High: {}/{}: {}'.format(rsicrossover_high,rsicrossover_low,rsiperiod, period_profit))
        period_netprofit.append((rsicrossover_high,rsicrossover_low,rsiperiod, period_profit))
        
    # Get the best profit parameters
    max_set = max(period_netprofit, key=lambda x:x[1])
    max_rsicrossover_high,max_rsicrossover_low,max_rsiperiod, max_profit = max_set
    
    # Add it to the dictionary
    print('\t\tBest is: {},{},{} with {} profit'.format(max_rsicrossover_high,max_rsicrossover_low,max_rsiperiod, max_profit))
    period['best_insample_param'] = (max_rsicrossover_high,max_rsicrossover_low,max_rsiperiod)
    period['best_insample_profit'] = max_profit

## Walkforward Efficiency

In [ ]:
def days_from_date_tuple(mytuple):
    """Get days between dates to annualize"""
    days_start = datetime.datetime.strptime(mytuple[0], '%Y-%m-%d')
    days_end = datetime.datetime.strptime(mytuple[1], '%Y-%m-%d')
    days = (days_end - days_start).days
    return days

# Get in-sample and out-of-sample best parameters with correct pre-allocation
all_walkforward_efficiency = []
for period in periods:
    # Get the insample time period
    oos_start = period['oos'][0]
    oos_end = period['oos'][1]
    insample_profit = period['best_insample_profit']
    print('Out-of-Sample Period: {} - {}'.format(oos_start, oos_end))
    
    # Find the set of results that has the same best parameters
    target_result = None
    for result in results:
        r = result[0].analyzers.decisive.get_analysis()
        if r['params'] == period['best_insample_param']:
            target_result = r
            break
            
    # Get the equity curve of this best parameter
    target_df = target_result['equitydf']
    
    # Splice to the OOS date range only
    oos_profit = target_df[oos_start:oos_end]['net_profit'].sum()
    
    # Annualize the profit
    insample_days = days_from_date_tuple(period['is'])
    oos_days = days_from_date_tuple(period['oos'])

    insample_annual_profit = 365/insample_days * insample_profit
    oos_annual_profit = 365/oos_days * oos_profit
 
    # Calculate WFE
    period_wfe = insample_annual_profit * 100 / oos_annual_profit
    period['wfe'] = period_wfe
    all_walkforward_efficiency.append(period_wfe)
    
    print('\t Period Profit IS:{:0.2f} / OOS:{:0.2f}'.format(insample_profit, oos_profit))
    print('\t Annual Profit IS:{:0.2f} / OOS:{:0.2f}'.format(insample_annual_profit, oos_annual_profit))

    print('\t WFE: {:0.1f}%'.format(period_wfe))



wfe = sum(all_walkforward_efficiency) / len(all_walkforward_efficiency) 
print('Overall WFE: {:0.1f}%'.format(wfe))

## Walkforward Equity Curve

In [ ]:
# Create a Stratey
class WalkforwardStrategy(bt.Strategy):
    params = (       
        ('printlog', False),
        ('rsiperiod',14),
        ('rsicrossover_high',50),
        ('rsicrossover_low',50),
        ('printlog', False),
        ('live', False),
        ('walkforward', None),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low
        self.dataopen = self.datas[0].open
        
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        # Add a RSI indicator
        self.rsi = bt.indicators.RelativeStrengthIndex(self.datas[0], period=self.params.rsiperiod)
        
        self.buysig = bt.And(self.datahigh[0] > self.dataclose[-1], self.rsi>self.params.rsicrossover_high)
        self.sellsig = bt.Or(self.rsi<self.params.rsicrossover_low, self.dataclose[0] < self.datalow[-1])
        # Trim the indicators if we are running live, just need the current one
        if not self.params.live:
            pass        
        
        self.current_row = None
        self.wfrsi = []
        
        if self.params.walkforward:
            for run in self.params.walkforward:
                self.wfrsi.append({
                    'rsi': bt.indicators.RelativeStrengthIndex(self.datas[0], period=run['best_insample_param'][2]),
                    'test_period': run['oos'],
                    'train_param': run['best_insample_param'],
                }) 

        print(pformat(self.wfrsi))
        
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):

        # Walk-forward logic
        for row in self.wfrsi:
            start, end = row['test_period']
            period_start = datetime.datetime.strptime(start, '%Y-%m-%d')
            period_end = datetime.datetime.strptime(end, '%Y-%m-%d')
            if self.datetime.datetime() >= period_start and self.datetime.datetime() < period_end:
                self.current_row = row
        
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.buysig:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.sellsig:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
    def stop(self):
        self.log('(RSI Period: %2d, RSI Crossover Lo/High: %2d/%2d) Ending Value %.2f' %
                 (self.params.rsiperiod, self.params.rsicrossover_low, self.params.rsicrossover_high, self.broker.getvalue()), doprint=True)

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(WalkforwardStrategy, 
                    rsiperiod=14,
                    rsicrossover_high=50,
                    rsicrossover_low=50,                    
                    walkforward=periods,
                    live=False)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

# periods
start_date = periods[0]['oos'][0]
end_date = periods[-1]['oos'][0]

print('Start: {} End: {}'.format(start_date, end_date))
data = bt.feeds.PandasData(dataname = df[start_date:end_date])


# Add the Data Feed to Cerebro
cerebro.adddata(data)
cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run()

In [ ]:
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
r = results[0].analyzers.decisive.get_analysis()
cumsum = r['equitydf']['net_profit'].cumsum()

plt.figure(figsize=(10,5))
plt.plot(cumsum)
plt.title('Walkforward Equity Curve')
plt.xlabel('Trades')
plt.ylabel('Equity')

# cerebro.plot(volume=False, iplot=False)